In [12]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time


from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

from scipy import stats
from matplotlib import pyplot
from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")


import matplotlib
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')

import matplotlib.font_manager as fm


import seaborn as sns

table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/A5.pkl")
iHiVred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/iHiV4.pkl")
iHiV = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/iHiV5.pkl")
VHred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/VH4.pkl")
tableg = table.T
VH = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/VH5.pkl")
tableg = table.T
B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/B5.pkl")

Bdf = pd.DataFrame(B)

species_ones = [] 
for i in range(5):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(5):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

sp_list_provided = ['LP', 'LB', 'AP', 'AT', 'AO']

#### get all data -- design matrix, steady state abundances, and list of wells present


sst = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v5/Gould-etal-5species-relative-abundance-Oct22.pkl")
sstv3 = sst.copy()


from sklearn.metrics import r2_score


import matplotlib.patches as mpatches

from sklearn.ensemble import RandomForestRegressor


from sklearn.metrics import mean_absolute_percentage_error as mape



In [13]:
#csdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v5/rel_CS_WH.pkl")

#cv_splitsl = [3] 
#max_depthl = [2,5,10,15]
#ntrees = [100,200,300,400,500,1000]
#max_featuresl = ["auto","sqrt","log2"]

In [14]:
rdf = pd.read_pickle("RF-5sp-will-REL.pkl")

In [15]:
cv_splitsl = [3] 
max_depthl = [2,5,10,15]
ntrees = [100,200,300,400,500,1000]
max_featuresl = ["auto","sqrt","log2"]

In [16]:
rdf.head(3)

,species,max_depth,n_trees,Xtrain,Xtest,ypred,k-fold,max_features
0,0,2,100,"[3, 1, 5, 9, 0, 11, 2, 12, 8, 6]","[4, 10, 14, 7, 13, 15]","[0.6924295836549648, 0.5556821339335083, 0.499...",3,auto
1,0,2,100,"[4, 10, 14, 7, 13, 15, 11, 2, 12, 8, 6]","[3, 1, 5, 9, 0]","[0.43679070226378186, 0.4543008423809045, 0.44...",3,auto
2,0,2,100,"[4, 10, 14, 7, 13, 15, 3, 1, 5, 9, 0]","[11, 2, 12, 8, 6]","[0.44594506595405425, 0.745762991755684, 0.512...",3,auto


In [17]:
crossval = [] 
for m in cv_splitsl:
    samp = rdf[rdf["k-fold"]==m]
    for species in range(5):
        abunvec = pd.DataFrame(sstv3[species][species_ones[species]].values)
        sp1 = samp[samp["species"]==species]
        for md in max_depthl:
            rho1 = sp1[sp1["max_depth"]==md]
            for nest in ntrees: 
                ntre = rho1[rho1["n_trees"]==nest]
                for mfeats in max_featuresl:
                    mdf = ntre[ntre["max_features"]==mfeats]
                    r2 = [] 
                    yallbest  = []
                    yallact  = [] 
                    for iterii in range(m):
                        idx = mdf["Xtest"].iloc[iterii]
                        yact = abunvec.T[idx].T[0].values.astype(float)
                        ybest = mdf["ypred"].iloc[iterii]
                        ybestdf1 = pd.DataFrame(ybest)
                        ybestdf1[ybestdf1[0]<1e-10]=0
                        slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                        yallbest.append(ybestdf1[0].values)
                        yallact.append(yact)
                        rval = r2_score(yact,ybestdf1[0].values)
                        gop = 1/(2-rval)
                        mape1 = mape(yact,ybestdf1[0].values)
                        r2.append([gop,rval,mape1,slope,intercept,r_value**2])

                    r2df = pd.DataFrame(r2)
                    if m == 2:
                        ybestagg = [*yallbest[0],*yallbest[1]]
                        yactagg = [*yallact[0],*yallact[1]]
                    if m == 3:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                    if m == 5:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                    if m == 7:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                    #print(r2_score(yactagg,ybestagg))
                    #print(yactagg)
                    slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                    rval2 = r2_score(yactagg,ybestagg)
                    gop2 = 1/(2-rval2)
                    mean1 = r2df.mean().values[0]
                    mean2 = r2df.mean().values[1]
                    mean3 = r2df.mean().values[2]
                    mean4 = r2df.mean().values[3]
                    mean5 = r2df.mean().values[4]
                    mean6 = r2df.mean().values[5]
                    mape2 = mape(yactagg,ybestagg)
                    crossval.append([m,species,md,nest,mfeats,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])

In [19]:
yactagg

[0.31121341577360745,
 0.3930316535100277,
 0.2925493549919123,
 0.31222926962445713,
 0.9999850953062553,
 0.3325090270019124,
 0.3457469552858963,
 0.32894530322782267,
 0.32083122084874777,
 0.3812191932713649,
 0.4722545430356481,
 0.5873602809506023,
 0.7056665889233219,
 0.8866218278137661,
 0.39358628076468377,
 0.3455877470903599]

In [8]:
crossdf = pd.DataFrame(crossval,columns=["k-fold","sp","max_depth","trees","features","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [9]:
crossdf.head(4)

,k-fold,sp,max_depth,trees,features,nnseagg,r2agg,mapeagg,slagg,intagg,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,3,0,2,100,auto,0.579821,0.275329,0.185389,0.388243,0.323320,0.300792,break,0.565773,0.202541,0.186006,0.703721,0.164577,0.846091,"[0.5952327519651394, 0.6115552412915117, 0.423...","[0.6924295836549648, 0.5556821339335083, 0.499..."
1,3,0,2,100,sqrt,0.591210,0.308555,0.169908,0.274368,0.388045,0.315224,break,0.577501,0.261034,0.167375,0.476870,0.286360,0.803427,"[0.5952327519651394, 0.6115552412915117, 0.423...","[0.6489616840001493, 0.5321343607840777, 0.521..."
2,3,0,2,100,log2,0.599780,0.332721,0.161653,0.281627,0.378508,0.344045,break,0.604521,0.326870,0.158812,0.484834,0.276167,0.805704,"[0.5952327519651394, 0.6115552412915117, 0.423...","[0.6522090736520368, 0.5269074010097559, 0.499..."
3,3,0,2,200,auto,0.596917,0.324725,0.172817,0.380681,0.327929,0.331976,break,0.586647,0.271023,0.172936,0.673677,0.180888,0.845537,"[0.5952327519651394, 0.6115552412915117, 0.423...","[0.6725420847852646, 0.5571819475440356, 0.501..."


In [10]:
pd.to_pickle(crossdf,"RF-REL-analysis.pkl")
